In [2]:
import os
os.chdir('../')

In [3]:
%pwd

'c:\\ml_ops-pro'

In [4]:
import tensorflow as tf

In [ ]:
model = tf.keras.models.load_model('artifacts/training/model.h5')

In [7]:
os.environ['MLFLOW_TRACKING_URI'] ='https://dagshub.com/sharadnaik001/ml_ops-pro.mlflow/#/'

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [9]:
from src.CnnClasifier.constants import *
from src.CnnClasifier.utils.common import read_yaml, create_directories, save_json
from src.CnnClasifier import logging

class ConfiManeger:
    
    def __init__(
        self,
        config_filepath = path_of_config,
        params_filepath = path_of_params):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/sharadnaik001/ml_ops-pro.mlflow/#/",
            all_params=self.params,
            params_image_size=self.params.IMG_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config





In [ ]:
# !pip install python-box==4.0.3

In [10]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [ ]:
!pip install mlflow

In [ ]:
!pip install dagshub

In [11]:
import dagshub
dagshub.init(repo_owner='sharadnaik001', repo_name='ml_ops-pro', mlflow=True)


❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\rich\live.py:231: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=e9ca0a25-c585-4810-8d1d-843a5b3768da&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=2fe7c2009e941ddb5f13f9ae2f90d83e4d62d424a1929507eba8765026caa343




Accessing as sharadnaik001

Initialized MLflow to track repo "sharadnaik001/ml_ops-pro"

Repository sharadnaik001/ml_ops-pro initialized!

In [13]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )



    @staticmethod
    def load_model(path: Path)-> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model  = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)



    def log_into_mlFlow(self):

       mlflow.set_registry_uri(self.config.mlflow_uri)
       tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
       mlflow.end_run()  # End any active run before starting a new one
   
       with mlflow.start_run():
           mlflow.log_params(self.config.all_params)
           mlflow.log_metrics({"loss": self.score[0], "accuracy": self.score[1]})
   
           if tracking_url_type_store != "file":
               # Register the model if not using the 'file' scheme
               try:
                   mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
               except mlflow.exceptions.MlflowException as e:
                   logging.warning(f"Model registration failed: {str(e)}")
           else:
               # Log model without registering if using a local file store
               mlflow.keras.log_model(self.model, "model")










In [14]:
# model.summary()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [15]:
try:
    confi = ConfiManeger()
    evl = confi.get_evaluation_config()
    eval = Evaluation(evl)
    eval.evaluation()
    eval.log_into_mlFlow()

except Exception as d:
    raise d

Found 139 images belonging to 2 classes.


c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


9/9 ━━━━━━━━━━━━━━━━━━━━ 32s 3s/step - accuracy: 1.0000 - loss: 0.0676


2024/11/02 16:59:43 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2024/11/02 17:00:24 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\Admin\AppData\Local\Temp\tmp37xxozfh\model, flavor: keras). Fall back to return ['keras==3.2.1']. Set logging level to DEBUG to see the full traceback. 
2024/11/02 17:00:24 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/11/02 17:05:49 INFO mlflow.tracking._tracking_service.client: 🏃 View run efficient-lynx-426 at: https://dagshub.com/sharadnaik001/ml_ops-pro.mlflow/#/experiments/0/runs/07a12a918079444b8283ef540ad4afd0.
2024/11/02 17:05:49 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/sharadnaik001/ml_ops-pro.mlflow/#/experiments/0.


KeyboardInterrupt: 

In [ ]:
tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
print(tracking_url_type_store)